In [ ]:
# Importamos las librerías
import numpy as np
import pandas as pd
import json             # Para pasar de json en formato string a json formato diccionario
import requests as re   # Para realizar peticiones bajo el protocolo HTTP (GET, POST...)
import time             # Para obtener timestamp a partir de fecha string. Timestamp es un número que representa los segundos desde 1970-01-01 00:00:00
import datetime         # Para pasar de timestamp a fecha (al contrario que el anterior paquete)
from tqdm import tqdm   # Para que el bucle salga bonito y pinte el porcentaje de avance
from IPython.display import clear_output  # Para ir borrando la salida del bucle y que no se acumulen barras de porcentaje

In [ ]:
# Esta celda sirve para autenticarse con tu cuenta de DRIVE. Al ejecutarla, aparecerá un enlace, inicias sesión y copias el código que te sale.
# A continuación, pegas el código en el campo que aparece aquí abajo.
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
# Posibles criptomonedas a elegir
cryptocurrencies = ['BTC', 'ETH', 'LTC', 'XRP', 'DOGE', 'BNB', 'ADA']   

# Elegimos la moneda según la posicion en la lista de arriba (0 es BTC, 1 es ETH...)
idx = 0       
                                                         
# Comprobamos que hemos elegido la criptomoneda deseada
print(f'La criptomoneda elegida es {cryptocurrencies[idx]}.')           

La criptomoneda elegida es BTC.


In [ ]:
# Cargamos el dataset
df_orig = pd.read_csv(f"your_data_path/dataBTC.csv")  

In [ ]:
df = df_orig.copy()
df

,Unnamed: 0,asset_id,time,open,close,high,low,volume,market_cap,url_shares,unique_url_shares,reddit_posts,reddit_posts_score,reddit_comments,reddit_comments_score,tweets,tweet_spam,tweet_followers,tweet_quotes,tweet_retweets,tweet_replies,tweet_favorites,tweet_sentiment1,tweet_sentiment2,tweet_sentiment3,tweet_sentiment4,tweet_sentiment5,tweet_sentiment_impact1,tweet_sentiment_impact2,tweet_sentiment_impact3,tweet_sentiment_impact4,tweet_sentiment_impact5,social_score,average_sentiment,sentiment_absolute,sentiment_relative,search_average,news,price_score,social_impact_score,correlation_rank,galaxy_score,volatility,alt_rank,alt_rank_30d,market_cap_rank,percent_change_24h_rank,volume_24h_rank,social_volume_24h_rank,social_score_24h_rank,medium,youtube,social_contributors,social_volume,price_btc,social_volume_global,social_dominance,percent_change_24h,market_cap_global,market_dominance
0,0,1,1523264400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,30.0,14.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,1523268000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,75.0,40.0,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,NaN,NaN,NaN,NaN,NaN,NaN
2,0,1,1523271600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,40.0,59.0,61.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82,NaN,NaN,NaN,NaN,NaN,NaN
3,0,1,1523275200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.0,63.0,77.0,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125,NaN,NaN,NaN,NaN,NaN,NaN
4,0,1,1523278800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.0,82.0,85.0,165.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27727,0,1,1626408000,31955.771162,31943.856539,32040.245094,31898.221292,9.583896e+08,5.998500e+11,1449.0,760.0,40.0,169.0,89.0,266.0,7297.0,3560.0,33242607.0,70.0,1062.0,347.0,4290.0,200.0,677.0,968.0,5091.0,361.0,760302.0,3125325.0,2169036.0,19599582.0,7594131.0,33250260.0,3.6,3.0,86.0,NaN,11.0,3.3,4.1,2.8,69.0,0.015417,107.0,46.0,1.0,1529.0,2.0,1.0,1.0,NaN,NaN,7181.0,8197,1.0,34989.0,23.427363,-2.356416,1.449605e+12,41.380241
27728,0,1,1626411600,31963.102758,31850.888552,31982.958414,31788.240008,8.773209e+08,5.977096e+11,2029.0,744.0,53.0,166.0,54.0,96.0,7918.0,3049.0,28097459.0,237.0,1029.0,482.0,3573.0,201.0,630.0,865.0,5962.0,260.0,594042.0,3807854.0,1762111.0,18912798.0,3025975.0,28105071.0,3.7,3.0,88.0,NaN,15.0,2.8,4.1,2.7,66.5,0.015380,94.0,48.0,1.0,1489.0,2.0,1.0,1.0,NaN,NaN,7514.0,8784,1.0,37873.0,23.193304,-2.330319,1.449593e+12,41.232919
27729,0,1,1626415200,31855.514373,31661.640045,31855.514373,31608.682377,8.427036e+08,5.944114e+11,2395.0,873.0,66.0,125.0,76.0,126.0,8606.0,3636.0,39044966.0,76.0,828.0,958.0,3715.0,175.0,664.0,1066.0,6358.0,343.0,1033115.0,4643675.0,2486288.0,22102141.0,8785324.0,39053189.0,3.7,3.0,89.0,NaN,11.0,1.8,4.1,2.7,61.5,0.015478,80.0,38.0,1.0,1464.0,2.0,1.0,1.0,NaN,NaN,8352.0,9632,1.0,37160.0,25.920344,-2.701550,1.438908e+12,41.309898
27730,0,1,1626418800,31652.956477,31808.055909,31808.785220,31624.060943,9.091977e+08,5.956591e+11,5265.0,1031.0,78.0,87.0,142.0,187.0,8819.0,6843.0,31664239.0,107.0,757.0,448.0,2841.0,281.0,997.0,996.0,6190.0,355.0,2366774.0,3024253.0,3528620.0,15333968.0,7414777.0,31673931.0,3.6,3.0,84.0,NaN,14.0,2.5,4.2,2.6,64.5,0.0

In [ ]:
last_date_ts = np.array(df['time'])[-1]
last_date = datetime.datetime.fromtimestamp(last_date)
last_date

datetime.datetime(2021, 7, 16, 8, 0)

In [ ]:
# Establecemos la contraseña de la API
APIKey = "your_api_key"

# Cada cuánto queremos datos. Puede ser 'hour' o 'day'
interval = 'hour'

# Número de registros que queremos (máximo 720)
data_points = '720'

# Fecha y hora a partir de la cual queremos los datos. Ejemplo de formato fecha y hora: 2021-07-02-16-58-42. 2 de julio 2021, las 16:58 horas, 42s 
start_date = '2017-05-13-00-00-00'

# Fecha y hora hasta la cual queremos datos (hasta ahora)
end_date = (datetime.datetime.now() + datetime.timedelta(hours = 2)).strftime('%Y-%m-%d-%H-%M-%S')

# Pasamos fecha y hora inicial a timestamp porque la API así lo requiere
start = int(time.mktime(datetime.datetime.strptime(start_date, "%Y-%m-%d-%H-%M-%S").timetuple()))

# Pasamos fecha y hora final a timestamp
end = int(time.mktime(datetime.datetime.strptime(end_date, "%Y-%m-%d-%H-%M-%S").timetuple()))

# Creamos la url con la que haremos la petición GET para obtener el JSON con los datos. Interpolamos en la url los parámetros anteriores
url = f"https://api.lunarcrush.com/v2?data=assets&key={APIKey}&symbol={cryptocurrencies[idx]}&data_points={data_points}&interval={interval}&start={start}&end={end}"

In [ ]:
# Creamos el dataframe con los datos (Ojo: Esta celda tarda unos 18 minutos en ejecutarse)
with tqdm(total=100) as pbar:                                                   # Esto solo sirve para poder ver el porcentaje de ejecución                           
  df = pd.DataFrame()                                                           # Creamos un df vacío sobre el cual iremos concatenando filas
  while start + 10800 < end:                                                    # Mientras no sobrepasemos la fecha fin, continuamos metiendo datos 
    request = json.loads(re.get(url).text)['data'][0]['timeSeries']             # Hacemos la petición
    for i in range(len(request)):                                               # Para cada JSON de los JSON's de 'request' (un total de 'data_points')
      df1 = pd.DataFrame(request[i], index = [0])                               # Para el JSON número 'i', pasamos ese JSON a Data Frame
      df = pd.concat([df, df1])                                                 # Lo añadimos al DataFrame 'grande', y así en cada iteración del bucle
    clear_output(wait=True)                                                     # Limpiamos la salida
    print(datetime.datetime.fromtimestamp(start))                               # Imprimimos la fecha que se está cargando actualmente (por ver si va bien)
    start = df.iloc[-1]['time']+5                                               # Actualizamos fecha start (sino el bucle sería infinito...)
    url = f"https://api.lunarcrush.com/v2?data=assets&key={APIKey}&symbol=BTC&data_points={data_points}&interval={interval}&start={start}&end={end}"
    pbar.update(np.round(1/38*100, 2))                                          # Hemos actualizado la url para la nueva petición. Actualizamos procentaje

  0%|          | 0/100 [00:00<?, ?it/s]


KeyError: ignored

In [ ]:
# Mostramos el Data Frame con los datos
df

,asset_id,time,open,close,high,low,volume,market_cap,url_shares,unique_url_shares,reddit_posts,reddit_posts_score,reddit_comments,reddit_comments_score,tweets,tweet_spam,tweet_followers,tweet_quotes,tweet_retweets,tweet_replies,tweet_favorites,tweet_sentiment1,tweet_sentiment2,tweet_sentiment3,tweet_sentiment4,tweet_sentiment5,tweet_sentiment_impact1,tweet_sentiment_impact2,tweet_sentiment_impact3,tweet_sentiment_impact4,tweet_sentiment_impact5,social_score,average_sentiment,sentiment_absolute,sentiment_relative,search_average,news,price_score,social_impact_score,correlation_rank,galaxy_score,volatility,alt_rank,alt_rank_30d,market_cap_rank,percent_change_24h_rank,volume_24h_rank,social_volume_24h_rank,social_score_24h_rank,medium,youtube,social_contributors,social_volume,price_btc,social_volume_global,social_dominance,percent_change_24h,market_cap_global,market_dominance
0,11,1524020400,None,None,None,None,None,None,None,None,1,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,None,NaN,NaN,NaN,NaN,NaN
0,11,1524024000,None,None,None,None,None,None,None,None,None,None,2,5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2,None,NaN,NaN,NaN,NaN,NaN
0,11,1524027600,None,None,None,None,None,None,None,None,None,None,1,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,None,NaN,NaN,NaN,NaN,NaN
0,11,1524045600,None,None,None,None,None,None,None,None,1,6,2,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,3,None,NaN,NaN,NaN,NaN,NaN
0,11,1524049200,None,None,None,None,None,None,None,None,None,None,3,4,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,3,None,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1,1626955200,31958.4,32233.6,32248.2,31862.7,1.34653e+09,603690755228,3483,1332,13,115,61,176,15358,6472,105538759,147,3284,1161,15505,415,2234,1187,10872,650,1983485,57770428,4816680,30404137,10584126,105562630,3.6,3,81,None,36,3.3,3.9,2.9,68.5,0.0288171,47,97,1,943,2,1,1,None,None,14196,16800,1,60752.0,27.653411,2.345428,1.435209e+12,42.062927
0,1,1626958800,32226.2,32104,32238.6,32090.8,8.64226e+08,602424064318,3039,1344,158,966,118,205,14802,5935,100411629,297,2933,2275,16753,456,1496,1410,10869,571,7070752,30063457,11282576,44308149,7708953,100438097,3.6,3,85,None,24,3,3.9,3.1,68,0.0287029,44,97,1,957,2,1,1,None,None,13972,16446,1,59900.0,27.455760,1.782553,1.429459e+12,42.143511
0,1,1626962400,32104.9,32308.6,32313.8,32058.3,1.07367e+09,605305689834,2704,1297,84,123,129,165,15681,5993,88841451,463,5814,3005,29209,441,1549,1605,11601,485,13825424,22243187,7251267,37441309,8118755,88882934,3.6,3,86,None,28,3.3,3.9,3,69,0.0287164,48,97,1,954,2,1,1,None,None,14245,17219,1,61081.0,28.190436,1.387773,1.437691e+12,42.102621
0,1,1626966000,32314.3,32558.1,32620.1,32292.2,1.4448e+09,609773252655,3664,1085,6,6,2,2,16807,8444,85770765,255,4356,2318,19223,483,1603,1929,12133,659,9503029,10136698,7887406,48507070,9762714,85800581,3.6,3,86,None,28,3.6,3.9,3.1,71,0.0289107,50,95,1,936,2,1,1,None,1,15820,17928,1,59981.0,29.889465,1.822246,1.450442e+12,42.040516


In [ ]:
# Veamos si ha cogido los datos más actualizados
print('El último registro del data frame es: {}'.format(datetime.datetime.fromtimestamp(df.iloc[-1]['time']+7200)))

In [ ]:
# Por defecto, el .csv con los datos se guardará en tu drive, en la carpeta raíz "drive/My Drive/"
# Pon COLAB = True si quieres guardar el .csv en la carpeta compartida de TFM. En este caso, se sobreescribirá lo anterior. Así que ojito :)
COLAB = False

file = f'data{cryptocurrencies[idx]}_prueba.csv'

df.to_csv(file)

if COLAB:
  !cp {file} "drive/My Drive/TFM MBIT/Data"
else:
  !cp {file} "drive/My Drive"